In [7]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def normalize_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" not in filetype:
        df = pd.read_excel(folder+filename)
        filename = filename.split(".")[0]+".csv"
        df.to_csv (folder+filename, index = None, header=True)
    
    df = pd.DataFrame([])
    df = pd.read_csv(folder+filename, encoding = "utf-8",sep='delimiter')
    df = df.iloc[:,0].str.split(",",expand=True)



    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:] #row,col
        df_results = df_results.replace("",0)
        df_results = df_results.astype(int)
        df_results.columns = new_header
        df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        df.iloc[index+2:index+18,1:] = df_results_norm
        print("processed...",i)
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1])
    return(df)

def zscore_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        df = df.iloc[:,0].str.split(",",expand=True)
    else:
        df = pd.read_csv(folder+filename)
        
    df_initial = df
    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    normalized_dataframe = pd.DataFrame([])
    # print("DF",df, "INDEX",df_results_index[0])
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:] #row,col
        # print("i and index",i,index, dateset_name, df_results)
        df_results = df_results.replace("",0)
        df_results = df_results.astype(int)

        df_results.columns = new_header
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        
        df_mean = df_results_norm.stack().mean()
        df_std = df_results_norm.stack().std()
        df_results_zscore = (df_results_norm - df_mean)/df_std
        df.iloc[index+2:index+18,1:] = df_results_zscore
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_zscore."+filename.split(".")[-1])

    return(df)


repo = git.Repo('.', search_parent_directories=True)
folder = repo.working_tree_dir +"\\yeast-data\\"

plate_list = os.listdir(folder)
print(plate_list,folder)

for file in plate_list:
    if "plate-data" in file:
        normalize_plate384(folder,file)
        zscore_plate384(folder,file)

['completed', 'plate-data-ReFRAME PGAP3_Plate 28.csv', 'plate-data-ReFRAME PGAP3_Plate 28.xlsx'] C:\Users\vandu\Desktop\vancanwin\yeast-data\
C:\Users\vandu\Desktop\vancanwin\yeast-data\plate-data-ReFRAME PGAP3_Plate 28.csv
processed... 0
C:\Users\vandu\Desktop\vancanwin\yeast-data\plate-data-ReFRAME PGAP3_Plate 28.csv
C:\Users\vandu\Desktop\vancanwin\yeast-data\plate-data-ReFRAME PGAP3_Plate 28.xlsx
processed... 0
C:\Users\vandu\Desktop\vancanwin\yeast-data\plate-data-ReFRAME PGAP3_Plate 28.xlsx


C:\Users\vandu\AppData\Local\Temp\ipykernel_12696\1506111168.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, encoding = "utf-8",sep='delimiter')
C:\Users\vandu\AppData\Local\Temp\ipykernel_12696\1506111168.py:50: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
C:\Users\vandu\AppData\Local\Temp\ipykernel_12696\1506111168.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as 

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte